## run this cell before starting spark cluster connection (corrects module error)

In [2]:
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [4]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

### select the data

In [5]:
data =  spark.read.json("/user/sprenner/forest_forward_selection_data_indexed_hashed.json")

In [6]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator




assembler = VectorAssembler(inputCols=["accountHash","clientStateHash","datasetHash","datasetScopeHash","eventTypeHash","eventVersionHash","filenameHash", \
                                       "filesize_double","hostnameHash","protocolHash","remoteSiteHash","scopeHash","traceIdHash","traceIpHash","traceTimeentryUnix", \
                                       "transferEnd","transferStart","uuidHash"], outputCol="features")
output = assembler.transform(data)
output = output.select("class", "features")
        
train_data, test_data = output.randomSplit([.8,.2],seed=1234)
rf = RandomForestClassifier(labelCol='class', featuresCol='features',numTrees=100)
model = rf.fit(train_data)
predictions = model.transform(test_data)

evaluator = MulticlassClassificationEvaluator(labelCol="class", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))


print(str(model.featureImportances))

Test Error = 0.529585
(18,[0,1,2,3,5,7,8,9,10,11,13,14,15,16,17],[0.141427292651,0.000135098503167,0.0555161190641,0.151706012934,0.00227231693831,0.0541725886296,0.0841491097171,0.000285943557922,0.0044444873045,0.152366696327,0.148704727455,0.0590305926541,0.0470611842856,0.0675889544275,0.0311388755517])


In [30]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import copy


# starts forward selection
def forwardSelection(data, features, target):
    featureList = copy.deepcopy(features)
    selected = []
    #keep track what the current error is to minimize it
    referenceScore = None
    while len(featureList) > 0:
        best = getBestFeature(data, featureList, selected, target, referenceScore)
        if best is not None:
            referenceScore = best[1]
            selected.append(best[0])
            featureList.remove(best[0])
            print("Selected " + str(best[0]))
            print("Current Feature Set: " + str(selected))
        else:
            print("No further improvement with removing features, stopping selection.")
            break
    return selected

# selects feature which yields the largest improvement together with already selected features (include)    
def getBestFeature(data, features, included, target, referenceScore):
    print("Getting next best feature...")
    if len(features) == 0:
        return "No features passed."
    include = copy.deepcopy(included)
    bestFeature = None
    for feature in features:
        # Don't include target feature, would be the best predictor otherwise
        if feature == target:
            print("Skipping " + str(feature))
            continue
        print("Calculating score for column " + str(feature) + " including " + str(include))
        # temporarily add current feature to feature set
        include.append(feature)
        assembler = VectorAssembler(inputCols=include, outputCol="features")
        output = assembler.transform(data)
        output = output.select(target, "features")
        train_data, test_data = output.randomSplit([.8,.2],seed=1234)
        error = regressionSquaredError(target, train_data)
        # first iteration
        if bestFeature is None:
            # first iteration overall
            if referenceScore is None:
                print("First feature overall found: " + str(feature) + " with score " + str(error))
                bestFeature = (feature, error)
                # for first iteration overall referenceScore must be set here
                referenceScore = error
            # first iteration but already reference error available
            elif error < referenceScore:
                print("First feature for loop found: " + str(feature) + " with score " + str(error))
                bestFeature = (feature, error)
        # the error must be smaller than previous selection and smaller than with fewer features
        elif error < bestFeature[1] and error < referenceScore:
            bestFeature = (feature, error)
            print("Better feature found: " + str(feature) + " with score " + str(error))
        # remove temporary feature from feature set to start again with next feature
        include.remove(feature)
    return bestFeature
        
# performs linear regression and returns mean squared error      
def regressionSquaredError(label, data):
    lr = LinearRegression(labelCol=label, maxIter=10, regParam=0.3, elasticNetParam=0.8)
    linearModel = lr.fit(data)
    return linearModel.summary.rootMeanSquaredError

# performs linear regression and returns r2        
def regressionR2(label, data):
    lr = LinearRegression(labelCol=label, maxIter=10, regParam=0.3, elasticNetParam=0.8)
    linearModel = lr.fit(data)
    return linearModel.summary.r2

In [31]:
df_ps_final.schema.names

['accountHash',
 'clientStateHash',
 'datasetHash',
 'datasetScopeHash',
 'dataset_versionHash',
 'datatypeHash',
 'eventTypeHash',
 'eventVersionHash',
 'filenameHash',
 'filesize_double',
 'hits',
 'hostnameHash',
 'prod_stepHash',
 'projectHash',
 'protocolHash',
 'remoteSiteHash',
 'run_numberHash',
 'scopeHash',
 'stream_nameHash',
 'traceIdHash',
 'traceIpHash',
 'traceTimeentryUnix',
 'transferEnd',
 'transferStart',
 'uuidHash']

### Perform forward selection

In [32]:
selected_features = forwardSelection(df_ps_final, df_ps_final.schema.names, "hits")

Getting next best feature...
Calculating score for column accountHash including []
First feature overall found: accountHash with score 10279.0082429
Calculating score for column clientStateHash including []
Calculating score for column datasetHash including []
Better feature found: datasetHash with score 10265.8577577
Calculating score for column datasetScopeHash including []
Better feature found: datasetScopeHash with score 10005.9593602
Calculating score for column dataset_versionHash including []
Calculating score for column datatypeHash including []
Calculating score for column eventTypeHash including []
Calculating score for column eventVersionHash including []
Calculating score for column filenameHash including []
Calculating score for column filesize_double including []
Skipping hits
Calculating score for column hostnameHash including []
Calculating score for column prod_stepHash including []
Better feature found: prod_stepHash with score 9596.0582614
Calculating score for colum

Calculating score for column uuidHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash']
Better feature found: uuidHash with score 8863.31870073
Selected uuidHash
Current Feature Set: ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash']
Getting next best feature...
Calculating score for column accountHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash']
First feature for loop found: accountHash with score 8825.20314066
Calculating score for column clientStateHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash']
Calculating score for column datasetHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash']
Calculating score for column datasetScopeHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash']
Calculating score for column datatypeHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash']
Better feature found: datatypeHa

Calculating score for column run_numberHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash']
Calculating score for column scopeHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash']
Calculating score for column stream_nameHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash']
Calculating score for column traceIdHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash']
Calculating score for column transferEnd including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash']
Calculating score for column transferStart including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash']
Selected accountHash
Current Feature Set: ['prod_st

Calculating score for column datasetScopeHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash', 'accountHash', 'run_numberHash', 'datasetHash']
First feature for loop found: datasetScopeHash with score 8391.96017081
Calculating score for column datatypeHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash', 'accountHash', 'run_numberHash', 'datasetHash']
Better feature found: datatypeHash with score 8384.9034658
Calculating score for column eventTypeHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash', 'accountHash', 'run_numberHash', 'datasetHash']
Calculating score for column eventVersionHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash', 'accountHash', 'run_numberHash', 'datasetHash']
Calculating score for column filenam

Calculating score for column filesize_double including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash', 'accountHash', 'run_numberHash', 'datasetHash', 'datatypeHash', 'projectHash']
Better feature found: filesize_double with score 8366.39187002
Skipping hits
Calculating score for column protocolHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash', 'accountHash', 'run_numberHash', 'datasetHash', 'datatypeHash', 'projectHash']
Better feature found: protocolHash with score 8366.09341767
Calculating score for column remoteSiteHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash', 'accountHash', 'run_numberHash', 'datasetHash', 'datatypeHash', 'projectHash']
Calculating score for column scopeHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnam

Calculating score for column transferEnd including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash', 'accountHash', 'run_numberHash', 'datasetHash', 'datatypeHash', 'projectHash', 'protocolHash', 'remoteSiteHash']
Calculating score for column transferStart including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash', 'accountHash', 'run_numberHash', 'datasetHash', 'datatypeHash', 'projectHash', 'protocolHash', 'remoteSiteHash']
Selected stream_nameHash
Current Feature Set: ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash', 'accountHash', 'run_numberHash', 'datasetHash', 'datatypeHash', 'projectHash', 'protocolHash', 'remoteSiteHash', 'stream_nameHash']
Getting next best feature...
Calculating score for column clientStateHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 

Calculating score for column eventTypeHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash', 'accountHash', 'run_numberHash', 'datasetHash', 'datatypeHash', 'projectHash', 'protocolHash', 'remoteSiteHash', 'stream_nameHash', 'eventVersionHash', 'filesize_double']
Calculating score for column filenameHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash', 'accountHash', 'run_numberHash', 'datasetHash', 'datatypeHash', 'projectHash', 'protocolHash', 'remoteSiteHash', 'stream_nameHash', 'eventVersionHash', 'filesize_double']
First feature for loop found: filenameHash with score 8335.43291646
Skipping hits
Calculating score for column scopeHash including ['prod_stepHash', 'traceIpHash', 'dataset_versionHash', 'uuidHash', 'traceTimeentryUnix', 'hostnameHash', 'accountHash', 'run_numberHash', 'datasetHash', 'datatypeHash', 'projectHash', 'protocolHash', 'remoteSi

In [33]:
selected_features

['prod_stepHash',
 'traceIpHash',
 'dataset_versionHash',
 'uuidHash',
 'traceTimeentryUnix',
 'hostnameHash',
 'accountHash',
 'run_numberHash',
 'datasetHash',
 'datatypeHash',
 'projectHash',
 'protocolHash',
 'remoteSiteHash',
 'stream_nameHash',
 'eventVersionHash',
 'filesize_double',
 'traceIdHash',
 'filenameHash']